## Request and Inspect IWM-Options

The Russell 2000 ETF (IWM) is a popular instrument for conservative option strategies.

In this example, we select call options suitible for a neutral/ moderate baerish option selling strategy


First, load ib-ruby environment

In [1]:
require_relative '../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Define the base

In [2]:
iwm =  Stock.new( symbol: :iwm ).verify.first
puts iwm.as_table

┌───────┬────────┬─────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│       │ symbol │ con_id  │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞═══════╪════════╪═════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Stock │ IWM    │ 9579970 │  SMART   │        │            │      IWM      │       │        │   USD    │
└───────┴────────┴─────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘


In [3]:
iwm_market_price =  iwm.market_price
puts "IWM: Bars (raw-data): #{iwm.bars}  \n    -->  market_price #{iwm.misc}"


IWM: Bars (raw-data): [{:bid=>232.1, :ask=>232.11, :last=>232.1, :close=>231.89}]  
    -->  market_price {:delayed=>232.1}


* We are requesting market data in a demo-account without realtime data permissions.  
* Concurrent calls of `iwm.market_price` add further raw-data tuples to `iwm.bars`. However, no timestamp is saved.
         
If the request is fired before the market opening, the returned market price is the closing price of the previous day. 


In [4]:
iwm_options = iwm.otm_options(right: :call, count: 10)
puts "Out of the Money Strikes to consider:" 
puts iwm_options.keys.to_f.join ", "

Out of the Money Strikes to consider:
232.0, 232.5, 233.0, 234.0, 235.0, 236.0, 237.0, 237.5, 238.0, 239.0, 240.0


In [5]:
puts "Available Expiries: "
puts iwm_options[iwm_options.keys.max].as_table

Available Expiries: 
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ IWM    │ 481723564 │ NASDAQOM │ 20210716 │    100     │      IWM      │ call  │  240.0 │   USD    │
│ Option │ IWM    │ 460975807 │ NASDAQOM │ 20210820 │    100     │      IWM      │ call  │  240.0 │   USD    │
│ Option │ IWM    │ 449416668 │ NASDAQOM │ 20210917 │    100     │      IWM      │ call  │  240.0 │   USD    │
│ Option │ IWM    │ 477501007 │ NASDAQOM │ 20211119 │    100     │      IWM      │ call  │  240.0 │   USD    │
│ Option │ IWM    │ 449416741 │ NASDAQOM │ 20211217 │    100     │      IWM      │ call  │  240.0 │   USD    │
│ Option │ IWM    │ 449416763 │ NASDAQOM │ 20220121 │    100     │      IWM      │ call  │ 

In [6]:
puts iwm_options.map{|x,y| y.detect{|o| o.strike == iwm_options.keys[5]}}.compact.as_table

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ IWM    │ 481723518 │ NASDAQOM │ 20210716 │    100     │      IWM      │ call  │  236.0 │   USD    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘


Lets inspect these Options further.

The method _request_greeks_ requests Tickdata from the TWS-Server and collects the calculated greeks.

The parameter `thread: true` switches to asynchronous requests. The results are always stored in `Option.greek`,
which is an alias for the `OptionDetail`-Model. Thus calling `.as_table` on `Option,greek` prints the contents of this model. 


In [15]:
iwm_options.each do  |x,y| 
   # update contracts with greeks 
   y.map{|o| o.request_greeks( thread: true)}.join;
   # and print them
   puts y.compact.sort_by( &:expiry).greek.as_table  unless y.compact.size.zero?
end
nil   # suppress further output

┌───────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                    │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞═══════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ call  IWM 20210716  232.0 │    4.21 │       0.17 │     0.00 │    0.54 │    0.04 │    0.22 │   -0.09 │
│ call  IWM 20210820  232.0 │    7.35 │       0.19 │     0.00 │    0.53 │    0.02 │    0.36 │   -0.06 │
│ call  IWM 20210917  232.0 │    9.17 │       0.20 │     0.00 │    0.53 │    0.02 │    0.44 │   -0.05 │
│ call  IWM 20211119  232.0 │   12.08 │       0.20 │     0.48 │    0.53 │    0.01 │    0.58 │   -0.04 │
│ call  IWM 20211217  232.0 │   13.03 │       0.20 │     1.05 │    0.53 │    0.01 │    0.64 │   -0.04 │
│ call  IWM 20220121  232.0 │   14.19 │       0.20 │     1.05 │    0.53 │    0.01 │    0.70 │   -0.03 │
│ call  IWM 20220218  232.0 │   15.17 │       0.20 │     1.05 │ 

┌───────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                    │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞═══════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ call  IWM 20210716  238.0 │    1.50 │       0.16 │     0.00 │    0.28 │    0.04 │    0.20 │   -0.07 │
│ call  IWM 20210820  238.0 │    4.28 │       0.18 │     0.00 │    0.39 │    0.02 │    0.35 │   -0.06 │
│ call  IWM 20210917  238.0 │    6.03 │       0.19 │     0.00 │    0.42 │    0.02 │    0.43 │   -0.05 │
│ call  IWM 20211119  238.0 │    8.85 │       0.19 │     0.48 │    0.45 │    0.01 │    0.59 │   -0.04 │
│ call  IWM 20211217  238.0 │    9.85 │       0.19 │     1.05 │    0.45 │    0.01 │    0.64 │   -0.04 │
│ call  IWM 20220121  238.0 │   10.97 │       0.19 │     1.05 │    0.46 │    0.01 │    0.71 │   -0.03 │
│ call  IWM 20220218  238.0 │   12.00 │       0.20 │     1.05 │ 